# Individual scheduler for truck arrival smoothing 
Author Ilias Parmaksizoglou & Alessandro Bombelli

This notebook provided a quick tour on how to use the Individual scheduler for truck arrival smoothing tool (3.2). The needed input is :
* Matrix of incoming trucks arrivals with approximate ETA, chaperoning indicator and target destination
* Feasible jobs combinations as derived by Job Generation for CAVs tool

The output of this tool contains an exact arrival schedule for incoming truck arrivals

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import os

# Load Data

The two inputs are stored in the core folder, in the files "jobs_combined.csv" and "arrivals_combined.csv". The combined flag determines the method used to solve the PJS. Some further parameters are derived from the nature of truck arrivals, while some port parameters are set accoding to user's preference. Specifically the following parameters can be set as desired:

* w_pen : penalty factor for assignment outside of time window
* del_thres : Acceptable threshold of maximum incurred delay
* no_cavs : Number of CAVs

In [2]:
method = "combined"
core_dir = os.path.join(os.getcwd(),"core")
jobs_namefile = f"jobs_{method}.csv"
arrivals_namefile = f"arrivals_{method}.csv"

# Load Trucks and Companies and Events
possible_jobs = pd.read_csv(os.path.join(core_dir,jobs_namefile))
planned_arrivals = pd.read_csv(os.path.join(core_dir,arrivals_namefile))

# Base Parameters
no_destinations = len(set(planned_arrivals['Destination']))
destinations = [f"D{x}" for x in range(1,no_destinations+1)]

# User stated Parameters
time_window_len = 60
steps =  range(1,2*time_window_len)
no_cavs = 3
cavs = range(0,no_cavs)
del_thres = 15
w_pen = 3


In [3]:

window = 5
occupied = {} # Can pass occupied spots from previous time-windows here if need be

window_trucks = planned_arrivals.loc[planned_arrivals["Window"] == window]

multidict_input = {}
for i,row in window_trucks.iterrows():
    multidict_input[row[0]] = row[1],row[2],row[3],row[5]
trucks,t_win,target_dest,arrival,escort = gp.multidict(multidict_input)

multidict_input = {}
idx = 0
window_jobs = possible_jobs.loc[possible_jobs["W"] == window]
for i,row in window_jobs.iterrows():
    name_index = f"J_{idx}"
    multidict_input[name_index] = row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7]
    idx+=1
jobs,Sq,W,TC,ET,LT,En,Ex,F = gp.multidict(multidict_input)

N = {}
for j in jobs:
    N[j] = (len(Sq[j].split(",")))

In [4]:

model = gp.Model('PJS')

# Decision Variables
z = model.addVars(jobs,steps,cavs,vtype=GRB.BINARY,name="z")
x = model.addVars(jobs,steps,cavs,vtype=GRB.BINARY,name="x")
w = model.addVars(jobs,trucks,vtype=GRB.BINARY,name="w")
k = model.addVars(destinations,steps,cavs,vtype=GRB.BINARY,name="k")

# Penalty DVs
dv = model.addVars(trucks,vtype=GRB.INTEGER,name="dv")
wp = model.addVars(jobs,steps,cavs,vtype=GRB.INTEGER,name="wp")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-19


Constraint 2 is a typical set-partitioning constraint that enforces each truck $t$ to be assigned to only one job among the ones containing $t$. With $\mathcal{J}_t$, we mean the subset of jobs $\mathcal{J}_t \subseteq \mathcal{J}$ containing truck $t$. Constraint set 3 enforces that only job-truck assignments that contain the considered truck $t$ are considered.

In [5]:
# Constraint (1) 
for t in trucks:
    model.addConstr(sum(w[j,t] for j in jobs if t in Sq[j]) == 1)

# Constraint (2) 
for t in trucks:
    model.addConstr(sum(w[j,t] for j in jobs if t not in Sq[j]) == 0)


 Constraints 3 enforce that a CAV can start a job at most once inside the time-window, while constraints 4 enforce that the same CAV cannot perform more than one job at the same time. Note that constraint set 4 is not enforced for the fictitious CAV 0, this is taken care of by Constraint set 5. Constraint set 6 enforce that if a job is chosen, then exactly the number of trucks relating to this job should be selected. 

In [6]:
# Constraint (3)
for j in jobs:
    for c in cavs:
        model.addConstr(sum(z[j,s,c] for s in steps) <=1)

# Constraint (4) - (5)
for s in steps:
    for c in cavs:
        if c >= 1:
            model.addConstr(sum(z[j,s,c] for j in jobs) <=1)
        else:
            for y in destinations:
                model.addConstr(sum(z[j,s,0] for j in jobs if y in F[j].split(",")) <=1)

# Constraint (6)
for j in jobs:
    for s in steps:
        for c in cavs:
            model.addConstr((z[j,s,c] == 1) >> (sum(w[j,t] for t in trucks) == N[j])) # Fast


Constraint set 7 defines a lower bound on the start-time of job $j$ by forcing it to be greater or equal to $ET_j$. $ET_j$ is defined in the pre-processing stage when creating bundles and is the latest arrival time at the gate among all trucks in job $j$. $LT_j$ is computed in a similar fashion. Constraints 8 influence variables $x_{jsc}$ so that, if a job is active, its length in terms of time-steps is equal to $LT_j - ET_j$ as computed by the bundling algorithm. These constraints imply that, when jobs are selected, they can be performed exactly as previously planned in terms of duration, and no unforeseen additional delay is accounted for.

In [7]:
# Constraint (7)
for j in jobs:
    for s in steps:
        for c in cavs:
            model.addConstr(ET[j]-s*z[j,s,c] <= steps[-1]*(1-z[j,s,c]))

# Constraint (8)
for j in jobs:
    for s in steps:
        cur_steps = range(s,steps[-1]+1)
        for c in cavs:
            model.addConstr((z[j,s,c] == 1) >> (sum(x[j,s_prime,c] for s_prime in cur_steps) == LT[j]-ET[j])) # Fast

Constraints 9-10 are similar to constraints 4-5 and ensure that a single CAV does not execute multiple jobs simultaneously. Constraint set 11 links variables $z_{jsc}$ and $x_{jsc}$ by imposing that, if a certain $x_{jsc}$ is active, then one $z_{jsc}$ decision variable for the set of time-steps in the interval $\left[s-(LT_j-ET_j),s\right]$ should be unitary. This ensures that the job is still active at time-step $t$ (i.e., that $x_{jsc}$=1).

In [8]:
# Constraint (9) - (10)
for s in steps:
    for c in cavs:
        if c >= 1:
            model.addConstr(sum(x[j,s,c] for j in jobs) <=1)
        if c == 0:
            for y in destinations:
                model.addConstr(sum(x[j,s,c] for j in jobs if y in F[j].split(",")) <=1)

# Constraint (11)
for j in jobs:
    job_length = LT[j] - ET[j]
    for s in steps:
        for c in cavs:
            model.addConstr(sum(z[j, s_prime,c] for s_prime in steps if s_prime >s-job_length and s_prime <=s) == x[j,s,c])


Constraints 12 determine the occupancy of all destinations $y \in F^j$ if a specific $z_{jsc}$ is active by ``reserving" destination $y$ via decision variable $k_{ysc}$ in the time range $\left[s+En^j_i,s+Ex^j_i\right]$, when such destination is occupied by a truck served by CAV $c$ as part of job $j$.

In [9]:
# Constraint (12)
for j in jobs:
    for s in steps:
        for c in cavs:
            for idx,i in enumerate(Sq[j].split(",")):
                en = float(En[j].split(",")[idx])
                ex = float(Ex[j].split(",")[idx])
                duration = int(ex-en)
                start = int(s+en)
                stop = int(s+ex)
                if stop in steps:
                    model.addConstr((z[j,s,c] == 1) >> (sum(k[target_dest[i],s_prime,c] for s_prime in range(start,stop))==duration)) # Fast

Constraint set 13 ensures that at most one CAV can block a destination at a specific time-step. Constraint set 14 implies that if a destination is occupied at a specific time-step $s$ by a job handled by CAV $c$, then one such job should be active ($x_{jsc}$=1). 

In [10]:
# Constraint (13)
for s in steps:
    for y in destinations:
        model.addConstr(sum(k[y,s,c] for c in cavs) <=1)

# Constraint (14)
for y in destinations:
    for s in steps:
        for c in cavs:
            if (y,s,c) not in occupied.keys():
                model.addConstr(1-sum(x[j,s,c] for j in jobs if y in F[j].split(",")) <=steps[-1]*(1-k[y,s,c]))

Constraints 15 - 17 determine prevent non-singleton jobs from being assigned to CAV 0 (15), singleton jobs for trucks needing to be chaperoned to be assigned to CAV 0 (16), and assign singleton jobs for trucks not needing to be chaperoned to CAV 0 (17). Constraint 18 blocks selections of CAVs that are occupied from jobs relating to a previous time-window.

In [11]:
# Constraint (15) - (17)    
for j in jobs:
    seq_j = list(Sq[j].split(","))

    if len(seq_j) >= 2:
        model.addConstr(sum(z[j,s,0] for s in steps) == 0)
    
    if len(seq_j) == 1:
        if escort[seq_j[0]] == 1:
            model.addConstr(sum(z[j,s,0] for s in steps) == 0)
        else:
            model.addConstr(sum(z[j,s,0] for s in steps) == 1)

# Constraint (18)  
for i in occupied.keys():
    model.addConstr(k[i] >= occupied[i])

Constraints 19 determine the time deviation of truck $t$ from its arrival at the gate by setting $d_t=s \cdot z_{jsc} -A_t$ for the active ($w_{jt}$,$z_{jsc}$) combination of decision variables. If a truck is serviced exactly when it reached the gate, then $s \cdot z_{jsc} =A_t$ and hence $d_t$=0. Constraint 20 limits the maximum delay incurred by a truck at the gate to be below a threshold, which can be assigned by the port operator. In case the threshold leads to infeasibility, this is the parameter that we increase step-wise until a feasible solution is found. Constraint set 21 assigns an extra penalty to a job exceeding the current time-window proportionally to the amount of time-steps that the spillover is associated to. Note that the constraint is only defined for the subset of time-steps $\mathcal{S}^{+}$ that might activate such penalty. 

In [12]:
# Constraint (19) 
for t in trucks:
    for j in jobs:
        model.addConstr((w[j,t] ==1) >> (sum(s*z[j,s,c] for s in steps for c in cavs)-arrival[t]==dv[t]))

# Constraint (20) 
for t in trucks:
    model.addConstr(dv[t] <=del_thres)           

# Constraint (21) 
for j in jobs:
    for s in steps:
        for c in cavs:
            if s> (time_window_len):
                model.addConstr(wp[j,s,c] >= w_pen*(x[j,s,c]*s-time_window_len))

The Objective function defines the total time cost of movements for trucks in a job plus the associated collective waiting cost. The first term in the objective maps the discrepancy between the earliest starting time of a job and the actual starting time, and is multiplied by the number of trucks involved in the job (disutility at the gate). The second term maps whether a specific job exceeded the width of the current time-window via a penalty factor (disutility inside the port terminal). 

In [ ]:
model.setObjective(sum(z[j,s,c]*(TC[j]+ N[j]*(s-ET[j]))+wp[j,s,c] for j in jobs for s in steps for c in cavs)
                    ,GRB.MINIMIZE)

model.optimize()
obj = model.getObjective()

Save the occupied time-slots from next time-window to file as well as the final job selection and schedule

In [14]:
df = pd.DataFrame(columns = ["Truck","Arrival","Start","End","Deviation","CAV"])
for j in jobs:
    for s in steps:
        for c in cavs:
            if s> time_window_len:
                for y in destinations:
                    if k[y,s,c].x >=0.9999:
                        occupied[y,s-time_window_len,c]=1

for j in jobs:
    for t in trucks:
        if w[j,t].x >= 0.99:
            for s in steps:
                for c in cavs:
                    if z[j,s,c].x >= 0.99:
                        df2 = pd.DataFrame({"Truck" : [t],
                                                    "Arrival" : [(window-1)*time_window_len+arrival[t]],
                                                    "Start" : [(window-1)*time_window_len+s],
                                                    "End" : [(window-1)*time_window_len+s+LT[j]-ET[j]],
                                                    "Deviation" : [dv[t].x],
                                                    "CAV" : [c]
                                                    })
                        df = pd.concat([df, df2], ignore_index = True, axis = 0)

df.to_csv(os.path.join(core_dir,f"individual_schedule_{window}.csv"))